In [2]:
from tf_pwa.config_loader import ConfigLoader
from tf_pwa.amp.core import Particle
import extra_amp
import numpy as np
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from tf_pwa.cal_angle import cal_helicity_angle

2025-11-13 11:49:44.244529: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-13 11:49:44.245504: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-13 11:49:44.360429: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-13 11:49:48.346390: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation or

No module named 'uproot' you should install `uproot` correctly for using this module


In [3]:
res_names = ["X(3872)","X(3915)(0-)","chi(c2)(3930)","X(3940)(1+)","X(3993)","Psi(4040)","X(4300)","NR(0-)SPp","NR(1+)PSp","NR(0-)SPm","NR(1-)PPm","X0(2900)","X1(2900)"]

In [4]:
# Setting errors to 0

for i in range(len(res_names)):
    Resonance = res_names[i]

    with open("final_params.json", "r") as f:
        params = json.load(f)

    for key in list(params.get("error", {}).keys()):
            ks = str(key)
            if ks.endswith(('r', 'i')):
                if ks.endswith('r') and Resonance in ks:
                    params["error"][key] = 0.0
                else:
                    params["error"][key] = 0.0

    with open("final_params.json", "w") as fw:
        json.dump(params, fw, indent=2)

In [40]:
# Calculating amplitudes for each resonance with corresponding chain(s) set to 1

tf.random.set_seed(42)
con_one = ConfigLoader("config_a.yml")
p4_one = con_one.generate_phsp_p(1)
data_one = con_one.data.cal_angle(p4_one)
data_one["c"] = -np.ones((1,))

all_amps_one = []

for i in range(len(res_names)):
    Resonance = res_names[i]
    special = {"Dst->D0.pi_g_ls_0r": 1.0, "Dst->D0.pi_g_ls_0i": 0.0}

    with open("final_params.json", "r") as f:
        params = json.load(f)

    for key in list(params.get("value", {}).keys()):
        ks = str(key)
        if ks in special:
            params["value"][key] = special[ks]
            continue
        if ks.endswith(('r', 'i')):
            if ks.endswith('r') and Resonance in ks:
                params["value"][key] = 1.0
            else:
                params["value"][key] = 0.0

    with open("final_params.json", "w") as fw:
        json.dump(params, fw, indent=2)

    con_one.set_params("final_params.json")
    amp_model_one = con_one.get_amplitude()
    dg_one = amp_model_one.decay_group
    dg_one.set_used_res([i], only=True)
    amps_one = dg_one.get_amp(data_one).numpy().flatten()
    all_amps_one.append(amps_one)

print(all_amps_one)

for i in range(len(all_amps_one)):
    if all_amps_one[i] != 0:
        print("Found!", all_amps_one[i])
    else:
        print("Not found!", all_amps_one[i])

[array([0.19924532-0.00051605j]), array([0.03820619-0.03109762j]), array([0.03777471+0.03642083j]), array([0.+0.j]), array([-0.38050699+0.1226627j]), array([-0.02054878-0.05617016j]), array([-0.65754772-0.16224152j]), array([-0.08691402-0.03604088j]), array([0.+0.j]), array([0.10732879+0.j]), array([0.+0.08711448j]), array([0.35171363-0.06335187j]), array([-0.85513314+0.50243801j])]
Found! [0.19924532-0.00051605j]
Found! [0.03820619-0.03109762j]
Found! [0.03777471+0.03642083j]
Not found! [0.+0.j]
Found! [-0.38050699+0.1226627j]
Found! [-0.02054878-0.05617016j]
Found! [-0.65754772-0.16224152j]
Found! [-0.08691402-0.03604088j]
Not found! [0.+0.j]
Found! [0.10732879+0.j]
Found! [0.+0.08711448j]
Found! [0.35171363-0.06335187j]
Found! [-0.85513314+0.50243801j]


In [28]:
# Checking through a full simulation with original params

tf.random.set_seed(42)
con_one = ConfigLoader("config_a.yml")
con_one.set_params("final_params_full.json")
p4_one = con_one.generate_phsp_p(1)
print(p4_one)
data_one = con_one.data.cal_angle(p4_one)
data_one["c"] = np.ones((1,))

all_amps_one = []

amp_model_one = con_one.get_amplitude()
dg_one = amp_model_one.decay_group
amps_one = dg_one.get_amp(data_one).numpy().flatten()

for i in range(len(amps_one)):
    if amps_one[i] != 0:
        print("Found!")
        print(amps_one[i])
    else:
        print("Not found!")

{D: <tf.Tensor: shape=(1, 4), dtype=float64, numpy=array([[ 2.12973515, -0.10488107, -0.47447539, -0.89669044]])>, D0: <tf.Tensor: shape=(1, 4), dtype=float64, numpy=array([[ 2.05453428,  0.55169154, -0.07145529,  0.65882529]])>, K: <tf.Tensor: shape=(1, 4), dtype=float64, numpy=array([[ 0.93507897, -0.52015999,  0.56030212,  0.21482742]])>, pi: <tf.Tensor: shape=(1, 4), dtype=float64, numpy=array([[ 0.15999163,  0.07334953, -0.01437144,  0.02303774]])>}
Found!
(0.5562515131241264+0.48214150544609324j)


In [7]:
# Calculating amplitudes for each resonance with original params

con_one = ConfigLoader("config_a.yml")
con_one.set_params("final_params_full.json")
p4_one = con_one.generate_phsp_p(1)
data_one = con_one.data.cal_angle(p4_one)
data_one["c"] = np.ones((1,))

all_amps_one = []

for i in range(len(res_names)):
    Resonance = res_names[i]

    amp_model_one = con_one.get_amplitude()
    dg_one = amp_model_one.decay_group
    dg_one.set_used_res([i], only=True)
    amps_one = dg_one.get_amp(data_one).numpy().flatten()
    all_amps_one.append(amps_one)

print(all_amps_one)

for i in range(len(all_amps_one)):
    if all_amps_one[i] != 0:
        print("Found!")
    else:
        print("Not found!")

[array([-0.07207622+0.12899973j]), array([0.00427333-0.09307405j]), array([0.00400649-0.00852899j]), array([-0.01895916-0.1143822j]), array([-0.00821911+0.00999416j]), array([0.03518061-0.00444633j]), array([-0.04202466-0.02346973j]), array([-0.22567044-0.03234024j]), array([-0.16005341-0.00824008j]), array([-0.0758013-0.00041819j]), array([-0.05043794-0.1298387j]), array([0.+0.j]), array([0.-0.j])]
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Found!
Not found!
Not found!
